# Welcome to the CIFAR-10 KAN Performance check
So, the plan is, get the CIFAR-10 dataset, and try to make a KAN work on it, and more importantly, seeing how good of a performance can be achieved.

Some boiler plate stuff may be borrowed from the assignments, etc.

In [1]:
# Imports
import numpy as np
import pandas as pd
import torch
from torch import nn
from kan import *

from torchvision.datasets import CIFAR10

In [2]:
# Setup Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset Setup
cifar10_trainset = CIFAR10("./temp/", train=True, download=True)
cifar10_testset = CIFAR10("./temp/", train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Finalise dataset setup
train_x = torch.tensor(cifar10_trainset.data.reshape(50000, 32*32*3))
train_y = torch.tensor((pd.get_dummies(pd.Series(cifar10_trainset.targets)).map(lambda x: 1 if x == True else 0)).values)

test_x = torch.tensor(cifar10_testset.data.reshape(10000, 32*32*3))
test_y = torch.tensor((pd.get_dummies(pd.Series(cifar10_testset.targets)).map(lambda x: 1 if x == True else 0)).values)

# Setup the training data for the KAN
# https://kindxiaoming.github.io/pykan/Example/Example_4_classfication.html

dataset = {}
dataset['train_input'] = train_x.type(torch.float32).to(device)
dataset['test_input'] = test_x.type(torch.float32).to(device)
dataset['train_label'] = train_y.type(torch.float32).to(device)
dataset['test_label'] = test_y.type(torch.float32).to(device)

def test_acc():
    lim = 25
    dsize = len(dataset['test_input'])//lim
    res = 0
    for i in range(lim):
        v1 = torch.argmax(model(dataset['test_input'][i*dsize:(i+1)*dsize]), dim=1)
        v2 = torch.argmax(dataset['test_label'][i*dsize:(i+1)*dsize], dim=1)
        res += torch.sum(torch.eq(v1, v2))
        if i % 1 == 0:
            print(f'Completed {i+1} of {lim}')
    return (res / len(dataset['test_label']))

In [4]:
# Setup the pykan model:
model = KAN(width=[3*32*32, 100, 50, 10], grid=3, k=3, seed=42, device=device)

checkpoint directory created: ./model
saving model version 0.0


In [5]:
#Train & Test the KAN Model on CIFAR-10
# Inspired by this paper: https://arxiv.org/html/2411.05296v1
# https://arxiv.org/pdf/2404.19756 -- Page 7 for the bound
results = model.fit(dataset, opt="Adam", steps=10000, batch=400, lr=0.0001, loss_fn=torch.nn.CrossEntropyLoss());
print(f'Test Accuracy: {test_acc()}')

| train_loss: 1.07e+00 | test_loss: 1.22e+00 | reg: 3.21e+03 | : 100%|█| 10000/10000 [16:29<00:00, 1


saving model version 0.1
Completed 1 of 25
Completed 2 of 25
Completed 3 of 25
Completed 4 of 25
Completed 5 of 25
Completed 6 of 25
Completed 7 of 25
Completed 8 of 25
Completed 9 of 25
Completed 10 of 25
Completed 11 of 25
Completed 12 of 25
Completed 13 of 25
Completed 14 of 25
Completed 15 of 25
Completed 16 of 25
Completed 17 of 25
Completed 18 of 25
Completed 19 of 25
Completed 20 of 25
Completed 21 of 25
Completed 22 of 25
Completed 23 of 25
Completed 24 of 25
Completed 25 of 25
Test Accuracy: 0.5011999607086182
